Implementation of a quantum algorithm designed to check system specification consistency, developed by Mohammed Zidan, Ahmed M. Eisa, Montasir Qasymeh, and Mahmoud A. Ismail Shoman. [ [1] ]

[1]: https://ieeexplore-ieee-org.proxy.library.carleton.ca/document/10485497 "A Quantum Algorithm for System Specifications Verification"

In [ ]:
from qiskit import QuantumCircuit, QuantumRegister

Table 1. Truth table example for a consistent system with 3 variables and 2 function oracles (F1 = !X0 X2, F2 = !X0 !X1)
| X0 | X1 | X2 | F1  | F2  | Bool |
|----|----|----|----|----|----|
|  0 |  0 |  0 |  0  |  1  | False |
|  0 |  0 |  1 |  1  |  1  | True |
|  0 |  1 |  0 |  0  |  0  | False |
|  0 |  1 |  1 |  1  |  0  | False |
|  1 |  0 |  0 |  0  |  0  | False |
|  1 |  0 |  1 |  0  |  0  | False |
|  1 |  1 |  0 |  0  |  0  | False |
|  1 |  1 |  1 |  0  |  0  | False |

Table 2. Truth table example for an inconsistent system with 3 variables and 2 function oracles (F1 = X0 !X2, F2 = !X0 !X1)
| X0 | X1 | X2 | F1  | F2  | Bool |
|----|----|----|----|----|----|
|  0 |  0 |  0 |  0  |  1  | False |
|  0 |  0 |  1 |  0  |  1  | False |
|  0 |  1 |  0 |  0  |  0  | False |
|  0 |  1 |  1 |  0  |  0  | False |
|  1 |  0 |  0 |  1  |  0  | False |
|  1 |  0 |  1 |  0  |  0  | False |
|  1 |  1 |  0 |  1  |  0  | False |
|  1 |  1 |  1 |  0  |  0  | False |


In [ ]:
#Table 1 representation
consistent_system = {
    "terms" : {"0x1", "00x"},
    "var_count" : 3,
    "function_count" : 2
}

#Table 2 representation
inconsistent_system = {
    "terms" : {"1x0", "00x"},
    "var_count" : 3,
    "function_count" : 2
}

In [ ]:
def init__original_circuit(var_count: int, function_count: int) -> QuantumCircuit:
    qc_original = QuantumCircuit()

    qc_original.add_register(QuantumRegister(var_count, name="X"))

    for i in range(function_count):
        reg_name = f"F{i+1}"
        qc_original.add_register(QuantumRegister(1, name=reg_name))

    qc_original.add_register(QuantumRegister(1, name="aux"))

    qc_original.add_register(QuantumRegister(1, name="a1"))
    qc_original.add_register(QuantumRegister(1, name="a2"))

    return qc_original
    

In [ ]:
def apply_oracles(circuit: QuantumCircuit, terms, function_count):
    pass

References

[1] M. Zidan, A. M. Eisa, M. Qasymeh, and M. A. I. Shoman, “A quantum
algorithm for system specifications verification,” IEEE Internet of Things
Journal, vol. 11, no. 14, pp. 24 775–24 794, 2024.
```